những chỗ có thể sửa :

-số cluster

-xóa data xấu

-state : 3 states/âm vij

In [1]:
import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm
from sklearn.utils import check_random_state

In [2]:
def get_mfcc(file_path):
    y, sr = librosa.load(file_path) # read .wav file
    hop_length = math.floor(sr*0.010) # 10ms hop
    win_length = math.floor(sr*0.025) # 25ms frame
    
    #mfcc feature
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
    # delta mfcc feature 1st order and 2nd order
    delta1_mfcc = librosa.feature.delta(mfcc, order=1)
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)
    
    #energy feature:
    S, phase = librosa.magphase(librosa.stft(y, hop_length=hop_length, win_length=win_length)) # Spectrogram
    energy = librosa.feature.rms(S=S)
    #delta energy feature 1st order and 2nd order
    delta1_ener = librosa.feature.delta(energy, order=1)
    delta2_ener = librosa.feature.delta(energy, order=2)
    
    # X is 39 x T
    X = np.concatenate([mfcc, delta1_mfcc, delta2_mfcc, energy, delta1_ener, delta2_ener], axis=0) # O^r
    # return T x 39 (transpose of X)
    return X.T # hmmlearn use T x N matrix

def get_class_data(data_dir):
    files = os.listdir(data_dir)
    mfcc = [get_mfcc(os.path.join(data_dir,f)) for f in files if f.endswith(".wav")]
    print('data_dir: ', data_dir)
    print(f'mfcc.shape: {np.array(mfcc).shape}')
    return mfcc

def clustering(X, n_clusters=10):
    kmeans = KMeans(n_clusters=n_clusters, n_init=100, random_state=0, verbose=1)
    kmeans.fit(X)
    print("centers", kmeans.cluster_centers_.shape)
    return kmeans  


In [3]:
class_names = [ "yte", "dich", "tien", "cachly","nguoi" ,  "test_nguoi", "test_tien", "test_cachly", "test_dich" , "test_yte" ]
dataset = {}
for cname in class_names:
    print(f"Load {cname} dataset")
    dataset[cname] = get_class_data(os.path.join("full_data", cname))


Load yte dataset
data_dir:  full_data/yte
mfcc.shape: (67,)
Load dich dataset
data_dir:  full_data/dich
mfcc.shape: (49,)
Load tien dataset
data_dir:  full_data/tien
mfcc.shape: (60,)
Load cachly dataset
data_dir:  full_data/cachly
mfcc.shape: (75,)
Load nguoi dataset
data_dir:  full_data/nguoi
mfcc.shape: (60,)
Load test_nguoi dataset
data_dir:  full_data/test_nguoi
mfcc.shape: (15,)
Load test_tien dataset
data_dir:  full_data/test_tien
mfcc.shape: (16,)
Load test_cachly dataset
data_dir:  full_data/test_cachly
mfcc.shape: (23,)
Load test_dich dataset
data_dir:  full_data/test_dich
mfcc.shape: (13,)
Load test_yte dataset
data_dir:  full_data/test_yte
mfcc.shape: (20,)


In [4]:
# Get all vectors in the datasets
all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in dataset.items() if(not k.startswith("test"))], axis=0)
print("vectors", all_vectors.shape)

vectors (10861, 39)


In [5]:
# Run K-Means algorithm to get clusters
kmeans = clustering(all_vectors, 20)
print("centers", kmeans.cluster_centers_.shape)

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 33179286.932389714
start iteration
done sorting
end inner loop
Iteration 1, inertia 29580874.087030124
start iteration
done sorting
end inner loop
Iteration 2, inertia 28478437.609780405
start iteration
done sorting
end inner loop
Iteration 3, inertia 28141468.122327268
start iteration
done sorting
end inner loop
Iteration 4, inertia 27965716.673690137
start iteration
done sorting
end inner loop
Iteration 5, inertia 27859886.705884967
start iteration
done sorting
end inner loop
Iteration 6, inertia 27782471.247251444
start iteration
done sorting
end inner loop
Iteration 7, inertia 27716860.757767566
start iteration
done sorting
end inner loop
Iteration 8, inertia 27655113.951356973
start iteration
done sorting
end inner loop
Iteration 9, inertia 27616046.983793482
start iteration
done sorting
end inner loop
Iteration 10, inertia 27592283.11313069
start iteration
done sorting
end inner loop
Iteratio

In [6]:
models = {}
train_datas = {}
test_datas = {}
num_state = {}

In [7]:
def create_start(num_state, pre):    
    res = pre
    for i in range(num_state - len(pre)):
        res.append(0.0)
    return np.array(res)

In [8]:
def create_trans(num_state, pre):
    res = np.zeros((num_state, num_state))
    #Filling the diagonal
    for i in range(len(pre)):
        res[0][i] = pre[i]
        idx_row = 0
        idx_col = i
        while(True):
            idx_row += 1
            idx_col += 1
            if(idx_row == num_state or idx_col == num_state):
                break
            res[idx_row][idx_col] = pre[i]
    #Tracking invalid value:
    for i in range(num_state - 1, num_state - len(pre), -1):
        sum_re = 0.0
        for j in range(num_state - 1):
            sum_re += res[i][j]
        res[i][num_state - 1] = 1.0 - sum_re
    return res

In [9]:
def normalize_cus(a, axis=None):
    """
    Normalizes the input array so that it sums to 1.
    Parameters
    ----------
    a : array
        Non-normalized input data.
    axis : int
        Dimension along which normalization is performed.
    Notes
    -----
    Modifies the input **inplace**.
    """
    a_sum = a.sum(axis)
    if axis and a.ndim > 1:
        # Make sure we don't divide by zero.
        a_sum[a_sum == 0] = 1
        shape = list(a.shape)
        shape[axis] = 1
        a_sum.shape = shape

    a /= a_sum

## "dịch"

In [10]:
cname = 'dich' # dik

In [11]:
num_state[cname] = 6

print("Start matrix:")
test_start = create_start(num_state[cname], [0.7, 0.2, 0.1])
print(repr(test_start))
print()

print("Transition matrix:")
test_trans = create_trans(num_state[cname], [0.7, 0.2, 0.1])
print(repr(test_trans))

Start matrix:
array([0.7, 0.2, 0.1, 0. , 0. , 0. ])

Transition matrix:
array([[0.7, 0.2, 0.1, 0. , 0. , 0. ],
       [0. , 0.7, 0.2, 0.1, 0. , 0. ],
       [0. , 0. , 0.7, 0.2, 0.1, 0. ],
       [0. , 0. , 0. , 0.7, 0.2, 0.1],
       [0. , 0. , 0. , 0. , 0.7, 0.3],
       [0. , 0. , 0. , 0. , 0. , 1. ]])


In [12]:
#Setting "dich" model:

#Clustering 
train_datas[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])

#Initialization
hmm = hmmlearn.hmm.MultinomialHMM(
    n_components=num_state[cname], random_state=0, n_iter=1000, verbose=True,
    params='te',
    init_params='e'
)
hmm.startprob_= np.array([0.7,0.2,0.1,0.0,0.0,0.0])
hmm.transmat_= np.array(
    [[0.7, 0.2, 0.1, 0. , 0. , 0. ],
    [0. , 0.7, 0.2, 0.1, 0. , 0. ],
    [0. , 0. , 0.7, 0.2, 0.1, 0. ],
    [0. , 0. , 0. , 0.7, 0.2, 0.1],
    [0. , 0. , 0. , 0. , 0.7, 0.3],
    [0. , 0. , 0. , 0. , 0. , 1. ]])

#Fiting data and train
X = np.concatenate(train_datas[cname])
lengths = list([len(x) for x in train_datas[cname]])
print("training class", cname)
print(X.shape, lengths, len(lengths))
hmm.fit(X, lengths=lengths)
models[cname] = hmm

print("Finish training")

training class dich
(834, 1) [14, 14, 16, 19, 16, 14, 21, 14, 17, 24, 14, 19, 16, 23, 28, 17, 21, 14, 15, 23, 18, 18, 16, 16, 15, 16, 17, 16, 20, 16, 20, 20, 15, 14, 17, 14, 17, 20, 18, 12, 15, 17, 18, 18, 14, 15, 16, 14, 13] 49


         1       -2464.0679             +nan
         2       -1782.7613        +681.3067
         3       -1578.8778        +203.8835
         4       -1476.2630        +102.6148
         5       -1425.5254         +50.7376
         6       -1406.9948         +18.5307
         7       -1399.3412          +7.6535
         8       -1395.0453          +4.2960
         9       -1392.3463          +2.6990
        10       -1390.4458          +1.9005
        11       -1389.0113          +1.4345
        12       -1387.8146          +1.1966
        13       -1386.7022          +1.1124
        14       -1385.6018          +1.1004
        15       -1384.4953          +1.1065
        16       -1383.4013          +1.0940
        17       -1382.3628          +1.0385
        18       -1381.4278          +0.9349
        19       -1380.6294          +0.7984
        20       -1379.9730          +0.6563
        21       -1379.4302          +0.5429
        22       -1378.9102          +0.5200
        23

Finish training


        39       -1369.9597          +0.0386
        40       -1369.9302          +0.0294
        41       -1369.9076          +0.0226
        42       -1369.8901          +0.0175
        43       -1369.8765          +0.0136
        44       -1369.8659          +0.0106
        45       -1369.8575          +0.0084


## Tien


In [13]:
cname = "tien"  # t-ie-g

In [14]:
num_state[cname] = 11

print("Start matrix:")
test_start = create_start(num_state[cname], [0.7, 0.2, 0.1])
print(repr(test_start))
print()

print("Transition matrix:")
test_trans = create_trans(num_state[cname], [0.7, 0.2, 0.1])
print(repr(test_trans))

Start matrix:
array([0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])

Transition matrix:
array([[0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.3],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ]])


In [15]:
#Setting "tien" model:

#Clustering 
train_datas[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])

#Initialization
hmm = hmmlearn.hmm.MultinomialHMM(
    n_components=num_state[cname], random_state=0, n_iter=1000, verbose=True,
    params='te',
    init_params='e'
)

hmm.startprob_= np.array([0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])


hmm.transmat_= np.array([[0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.3],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ]])
#Fiting data and train
X = np.concatenate(train_datas[cname])
lengths = list([len(x) for x in train_datas[cname]])
print("training class", cname)
print(X.shape, lengths, len(lengths))
hmm.fit(X, lengths=lengths)
models[cname] = hmm

print("Finish training")

training class tien
(1638, 1) [21, 23, 34, 36, 37, 43, 21, 25, 18, 32, 32, 40, 25, 16, 24, 21, 33, 33, 22, 30, 28, 21, 31, 27, 21, 40, 19, 20, 43, 24, 24, 39, 29, 20, 23, 42, 26, 16, 19, 20, 21, 19, 24, 49, 29, 24, 23, 38, 25, 29, 33, 30, 20, 15, 24, 25, 27, 43, 15, 27] 60


         1       -4503.7742             +nan
         2       -2794.5017       +1709.2725
         3       -2518.2661        +276.2356
         4       -2410.9749        +107.2911
         5       -2343.3880         +67.5869
         6       -2269.2175         +74.1705
         7       -2179.6050         +89.6125
         8       -2125.2059         +54.3991
         9       -2100.8254         +24.3805
        10       -2083.2572         +17.5682
        11       -2067.4191         +15.8381
        12       -2054.4157         +13.0034
        13       -2048.9139          +5.5017
        14       -2046.8286          +2.0853
        15       -2045.4766          +1.3520
        16       -2044.2963          +1.1803
        17       -2043.3033          +0.9930
        18       -2042.6134          +0.6898
        19       -2042.1561          +0.4573
        20       -2041.8054          +0.3507
        21       -2041.4935          +0.3119
        22       -2041.2205          +0.2730
        23

Finish training


        35       -2040.3094          +0.0182
        36       -2040.2942          +0.0152
        37       -2040.2813          +0.0129
        38       -2040.2701          +0.0112
        39       -2040.2603          +0.0098


## y te

In [34]:
cname = "yte" # y _ t e

In [36]:
num_state[cname] = 12

print("Start matrix:")
test_start = create_start(num_state[cname], [0.7, 0.2, 0.1])
print(repr(test_start))
print()

print("Transition matrix:")
test_trans = create_trans(num_state[cname], [0.7, 0.2, 0.1])
print(repr(test_trans))

Start matrix:
array([0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])

Transition matrix:
array([[0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.3],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ]])


In [37]:
#Setting "y te" model:

#Clustering 
train_datas[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])

#Initialization
hmm = hmmlearn.hmm.MultinomialHMM(
    n_components=num_state[cname], random_state=0, n_iter=1000, verbose=True,
    params='te',
    init_params='e'
)

hmm.startprob_= np.array([0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])

hmm.transmat_= np.array([[0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.3],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ]])

#Fiting data and train
X = np.concatenate(train_datas[cname])
lengths = list([len(x) for x in train_datas[cname]])
print("training class", cname)
print(X.shape, lengths, len(lengths))
hmm.fit(X, lengths=lengths)
models[cname] = hmm

print("Finish training")

training class yte
(2954, 1) [43, 34, 42, 35, 44, 29, 47, 28, 40, 50, 38, 40, 39, 35, 37, 54, 29, 37, 54, 53, 31, 41, 61, 37, 33, 50, 42, 48, 62, 39, 44, 34, 31, 42, 49, 77, 45, 44, 61, 66, 31, 47, 49, 35, 79, 34, 39, 50, 41, 39, 44, 68, 32, 38, 36, 34, 53, 44, 53, 38, 49, 60, 43, 41, 48, 41, 43] 67


         1       -8751.2159             +nan
         2       -6421.1953       +2330.0206
         3       -5795.5585        +625.6368
         4       -5494.0459        +301.5127
         5       -5282.2346        +211.8113
         6       -5189.3267         +92.9079
         7       -5115.8678         +73.4589
         8       -5038.8663         +77.0015
         9       -5006.0122         +32.8541
        10       -4992.0865         +13.9258
        11       -4985.6443          +6.4421
        12       -4981.7604          +3.8839
        13       -4978.8553          +2.9052
        14       -4976.4366          +2.4186
        15       -4974.2763          +2.1604
        16       -4972.2267          +2.0496
        17       -4970.1638          +2.0629
        18       -4967.8543          +2.3095
        19       -4965.0493          +2.8050
        20       -4962.2663          +2.7830
        21       -4959.8018          +2.4645
        22       -4957.2770          +2.5248
        23

Finish training


        75       -4927.0908          +0.0093


## cach ly

In [19]:
cname = "cachly" # c a k _ l y

In [20]:
num_state[cname] = 12

print("Start matrix:")
test_start = create_start(num_state[cname], [0.7, 0.2, 0.1])
print(repr(test_start))
print()

print("Transition matrix:")
test_trans = create_trans(num_state[cname], [0.7, 0.2, 0.1])
print(repr(test_trans))

Start matrix:
array([0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])

Transition matrix:
array([[0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.3],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ]])


In [21]:
#Setting "cach ly" model:

#Clustering 
train_datas[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])

#Initialization
hmm = hmmlearn.hmm.MultinomialHMM(
    n_components=num_state[cname], random_state=0, n_iter=1000, verbose=True,
    params='te',
    init_params='e'
)

hmm.startprob_= np.array([0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])

hmm.transmat_= np.array([  [0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
                           [0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
                           [0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
                           [0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. , 0. ],
                           [0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. , 0. ],
                           [0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. , 0. ],
                           [0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. , 0. ],
                           [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. , 0. ],
                           [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1, 0. ],
                           [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.2, 0.1],
                           [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.7, 0.3],
                           [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. ]])

#Fiting data and train
X = np.concatenate(train_datas[cname])
lengths = list([len(x) for x in train_datas[cname]])
print("training class", cname)
print(X.shape, lengths, len(lengths))
hmm.fit(X, lengths=lengths)
models[cname] = hmm

print("Finish training")

training class cachly
(4002, 1) [82, 50, 56, 33, 61, 43, 65, 69, 67, 76, 49, 31, 61, 41, 90, 33, 39, 61, 49, 37, 61, 71, 38, 63, 51, 39, 68, 32, 65, 49, 34, 64, 36, 50, 43, 55, 89, 60, 59, 56, 70, 58, 61, 37, 30, 51, 68, 51, 60, 63, 59, 28, 61, 59, 28, 60, 29, 59, 64, 57, 64, 33, 89, 46, 35, 59, 56, 89, 35, 35, 59, 50, 54, 34, 35] 75


         1      -11683.6060             +nan
         2       -9399.1018       +2284.5041
         3       -8673.1064        +725.9955
         4       -8308.8639        +364.2425
         5       -8139.2119        +169.6520
         6       -8021.2862        +117.9257
         7       -7941.7110         +79.5752
         8       -7856.5659         +85.1451
         9       -7709.0926        +147.4733
        10       -7455.8677        +253.2249
        11       -7242.0959        +213.7719
        12       -7195.6453         +46.4506
        13       -7173.9691         +21.6761
        14       -7157.5345         +16.4346
        15       -7143.3395         +14.1950
        16       -7134.0733          +9.2663
        17       -7129.3187          +4.7546
        18       -7127.0234          +2.2953
        19       -7125.9681          +1.0553
        20       -7125.4298          +0.5384
        21       -7125.0939          +0.3359
        22       -7124.8384          +0.2556
        23

Finish training


        41       -7119.0390          +0.0077


## NGUOI


In [22]:
cname = "nguoi"

In [23]:
num_state[cname] = 6

print("Start matrix:")
test_start = create_start(num_state[cname], [0.7, 0.2, 0.1])
print(repr(test_start))
print()

print("Transition matrix:")
test_trans = create_trans(num_state[cname], [0.7, 0.2, 0.1])
print(repr(test_trans))

Start matrix:
array([0.7, 0.2, 0.1, 0. , 0. , 0. ])

Transition matrix:
array([[0.7, 0.2, 0.1, 0. , 0. , 0. ],
       [0. , 0.7, 0.2, 0.1, 0. , 0. ],
       [0. , 0. , 0.7, 0.2, 0.1, 0. ],
       [0. , 0. , 0. , 0.7, 0.2, 0.1],
       [0. , 0. , 0. , 0. , 0.7, 0.3],
       [0. , 0. , 0. , 0. , 0. , 1. ]])


In [24]:
#Setting "nguoi" model:

#Clustering 
train_datas[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])

#Initialization
hmm = hmmlearn.hmm.MultinomialHMM(
    n_components=num_state[cname], random_state=0, n_iter=1000, verbose=True,
    params='te',
    init_params='e'
)

hmm.startprob_= np.array([0.7,0.2,0.1,0.0,0.0,0.0])
hmm.transmat_= np.array(
    [[0.7, 0.2, 0.1, 0. , 0. , 0. ],
    [0. , 0.7, 0.2, 0.1, 0. , 0. ],
    [0. , 0. , 0.7, 0.2, 0.1, 0. ],
    [0. , 0. , 0. , 0.7, 0.2, 0.1],
    [0. , 0. , 0. , 0. , 0.7, 0.3],
    [0. , 0. , 0. , 0. , 0. , 1. ]])


#Fiting data and train
X = np.concatenate(train_datas[cname])
lengths = list([len(x) for x in train_datas[cname]])
print("training class", cname)
print(X.shape, lengths, len(lengths))
hmm.fit(X, lengths=lengths)
models[cname] = hmm

print("Finish training")

         1       -4315.4981             +nan
         2       -2829.6487       +1485.8494


training class nguoi
(1433, 1) [18, 19, 26, 22, 16, 25, 24, 23, 18, 19, 31, 27, 22, 17, 15, 24, 20, 15, 11, 26, 23, 23, 43, 26, 26, 19, 35, 26, 16, 21, 30, 21, 17, 25, 23, 22, 20, 36, 28, 24, 25, 30, 18, 21, 22, 13, 27, 24, 27, 35, 23, 16, 21, 34, 18, 19, 21, 58, 37, 22] 60


         3       -2568.6296        +261.0190
         4       -2479.6582         +88.9715
         5       -2451.3900         +28.2682
         6       -2432.0726         +19.3174
         7       -2423.1501          +8.9225
         8       -2420.1940          +2.9562
         9       -2419.1195          +1.0744
        10       -2418.5855          +0.5340
        11       -2418.1790          +0.4065
        12       -2417.6961          +0.4830
        13       -2416.9134          +0.7826
        14       -2415.7136          +1.1999
        15       -2414.3237          +1.3898
        16       -2412.8619          +1.4618
        17       -2411.2660          +1.5959
        18       -2409.5915          +1.6745
        19       -2407.9957          +1.5958
        20       -2406.4416          +1.5541
        21       -2404.5035          +1.9381
        22       -2401.0726          +3.4309
        23       -2393.4822          +7.5904
        24       -2379.5166         +13.9656
        25

Finish training


        45       -2357.0045          +0.0103
        46       -2356.9957          +0.0088


## Setting testing dataset:

In [25]:
testing_set = {}
for k, v in train_datas.items():
    testing_set[k] = list([kmeans.predict(v).reshape(-1,1) for v in dataset["test_"+k]])

In [47]:
import joblib

def save_mod(cname, model):
  joblib.dump(model, f"{cname}.pkl")
  print(f"Save model {cname} success")
    

#Save kmeans:
save_mod("kmeans_ver2", kmeans,)
#Save hmm
for cname in class_names:
  save_mod(cname +  "_ver2", models[cname])

Save model kmeans_ver2 success
Save model tien_ver2 success
Save model cachly_ver2 success
Save model dich_ver2 success
Save model yte_ver2 success
Save model nguoi_ver2 success


In [48]:
class_names = ["tien", "cachly",  "dich", "yte", "nguoi"]

In [49]:
for name in class_names:
    print(name)
    # models[name] = joblib.load(f"{name}.pkl")
#     print(models[name].emissionprob_.shape)
    np.set_printoptions(precision=3, suppress = True)
    print('\nStart probability:\n',models[name].startprob_)
    print('\nTransition matrix:\n', models[name].transmat_)
    print('*'*60, '\n')

tien

Start probability:
 [0.7 0.2 0.1 0.  0.  0.  0.  0.  0.  0.  0. ]

Transition matrix:
 [[0.67  0.33  0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.341 0.659 0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.004 0.885 0.111 0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.618 0.291 0.091 0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.787 0.037 0.177 0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.631 0.029 0.34  0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.64  0.342 0.018 0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.468 0.468 0.064 0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.504 0.496 0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.859 0.141]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.   ]]
************************************************************ 

cachly

Start probability:
 [0.7 0.2 0.1 0.  0.  0.  0.  0.  0.  0.  0.  0. ]

Trans

In [38]:
num_true = {}
num_false = {}

#Initialization:
for name in class_names:
    num_true[name] = 0
    num_false[name] = 0

#Testing:
for true_cname in class_names:
    print(f"Testing word {true_cname}:")
    for O in testing_set[true_cname]:
#         print("???")
#         print(O[10])
        scores = {cname : model.score(O, [len(O)]) for cname, model in models.items()}
        pred_name = max(scores, key=lambda key: scores[key])                
        if(pred_name == true_cname):
            num_true[true_cname] += 1
        else:
            num_false[true_cname] += 1            
        print(f"Prediction word: {pred_name},", f"Status: {pred_name == true_cname},",scores)
        print()

Testing word tien:
Prediction word: tien, Status: True, {'dich': -59.74811354805736, 'tien': -43.025674301251854, 'yte': -210.45735993516917, 'cachly': -48.92021093132415, 'nguoi': -59.2602296177007}

Prediction word: tien, Status: True, {'dich': -50.91223292477808, 'tien': -27.437270354821347, 'yte': -108.82137274460455, 'cachly': -70.23496035149198, 'nguoi': -221.23211773571865}

Prediction word: tien, Status: True, {'dich': -33.488363011589094, 'tien': -29.09769335362277, 'yte': -53.53707805208569, 'cachly': -58.30068116809674, 'nguoi': -54.72563553783333}

Prediction word: tien, Status: True, {'dich': -42.5655232743161, 'tien': -32.64280037380259, 'yte': -52.084233468055444, 'cachly': -43.29429308031519, 'nguoi': -70.76185034032203}

Prediction word: tien, Status: True, {'dich': -48.83099363728059, 'tien': -32.335661467764886, 'yte': -49.63134961835082, 'cachly': -51.466657149264144, 'nguoi': -inf}

Prediction word: cachly, Status: False, {'dich': -94.76860813123218, 'tien': -48.65

In [39]:
print("Test Result: ")
for name in class_names:
    acc =  100*num_true[name]/(num_true[name]+num_false[name])
    print(f"Acc on word \"{name}\": {acc}")
    print(f"Correct Rate: {num_true[name]}\\{num_true[name]+num_false[name]} ")
    print()
sum_true = sum(num_true.values())
sum_false = sum(num_false.values())
print(f"Overall Acc: {sum_true*100 / (sum_true+sum_false)}")

Test Result: 
Acc on word "tien": 87.5
Correct Rate: 14\16 

Acc on word "cachly": 95.65217391304348
Correct Rate: 22\23 

Acc on word "dich": 100.0
Correct Rate: 13\13 

Acc on word "yte": 85.0
Correct Rate: 17\20 

Acc on word "nguoi": 86.66666666666667
Correct Rate: 13\15 

Overall Acc: 90.80459770114942


# Test thử file thu từ mic

In [41]:
print(f"Load mic dataset")
mic_classes = ['test_mic_dich','test_mic_cachly', 'test_mic_yte', 'test_mic_tien', 'test_mic_nguoi']
for c_name in mic_classes:
    print(f"Loading {c_name} data")
    dataset[c_name] = get_class_data(os.path.join("full_data", c_name)) 


Load mic dataset
Loading test_mic_dich data
data_dir:  full_data/test_mic_dich
mfcc.shape: (4,)
Loading test_mic_cachly data
data_dir:  full_data/test_mic_cachly
mfcc.shape: (5,)
Loading test_mic_yte data
data_dir:  full_data/test_mic_yte
mfcc.shape: (7,)
Loading test_mic_tien data
data_dir:  full_data/test_mic_tien
mfcc.shape: (7,)
Loading test_mic_nguoi data
data_dir:  full_data/test_mic_nguoi
mfcc.shape: (5,)


In [42]:
mic_set = {}

In [43]:
total_true = 0
total_false = 0
mic_set = {}
#Clustering 
for c_name in mic_classes:
    mic_set[c_name] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[c_name]])
    
for c_name in mic_classes:
    print('***\n', c_name)
    for O in mic_set[c_name]:
        scores = {cname : model.score(O, [len(O)]) for cname, model in models.items()}
        print(scores)
        
        pred_name = max(scores, key=lambda key: scores[key])   
#         print('name: ', pred_name, '\n', '*',c_name[c_name.rfind('_')+1:],'*')
        if(pred_name == c_name[c_name.rfind('_')+1:].strip() ):
            total_true += 1
            print("\nPrediction is correct ! ")
        else:
            total_false += 1
            print("\nFalse prediction !")
        print()
 
print(f"Acc on validation data: {(total_true*100) / (total_true+total_false)}")

***
 test_mic_dich
{'dich': -54.01918386714371, 'tien': -inf, 'yte': -83.33352931829205, 'cachly': -inf, 'nguoi': -102.93229779690165}

Prediction is correct ! 

{'dich': -45.30323061254098, 'tien': -inf, 'yte': -42.02341181507555, 'cachly': -inf, 'nguoi': -45.787173251019695}

False prediction !

{'dich': -64.279506719042, 'tien': -inf, 'yte': -71.87072673460447, 'cachly': -148.67904706470483, 'nguoi': -111.4292098221076}

Prediction is correct ! 

{'dich': -72.70597298588821, 'tien': -inf, 'yte': -80.59261411991753, 'cachly': -202.45739541143024, 'nguoi': -inf}

Prediction is correct ! 

***
 test_mic_cachly
{'dich': -405.70995149808016, 'tien': -1519.2970759681155, 'yte': -309.5358948756527, 'cachly': -93.50029542253546, 'nguoi': -inf}

Prediction is correct ! 

{'dich': -517.2282446764863, 'tien': -inf, 'yte': -217.23295717966727, 'cachly': -147.03742370874718, 'nguoi': -209.52030773774874}

Prediction is correct ! 

{'dich': -428.00660152033237, 'tien': -5057.0278147225945, 'yte':